In [67]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split

In [68]:
save_data = False
tmp_data = True

In [104]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/"

data_subdir = "Dstlnu_Hc_corr_BsigX_separation_dataRun1/"   
root_subdir = "axheim_data2_MC14_100kEvts/"   

root_path = nfs_path + "createBranchSeparatorData/" + root_subdir

In [70]:
merged = "merged_"
if tmp_data:
    merged += "tmp_"

In [71]:
fileY4S = uproot.open(root_path + merged + "DXtagDstl.root:variables")

In [72]:
names = ["gammas","electrons","pions","kaons","muons"]
dfs = []
for name in names:
    filename = root_path + merged + "{}.root:variables".format(name)
    print(filename)
    tmpFileFSPs = uproot.open(filename)
    df_tmp = tmpFileFSPs.arrays(library="pd")
    dfs.append(df_tmp)

/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/createBranchSeparatorData/axheim_data2_MC14_100kEvts/merged_tmp_gammas.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/createBranchSeparatorData/axheim_data2_MC14_100kEvts/merged_tmp_electrons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/createBranchSeparatorData/axheim_data2_MC14_100kEvts/merged_tmp_pions.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/createBranchSeparatorData/axheim_data2_MC14_100kEvts/merged_tmp_kaons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/createBranchSeparatorData/axheim_data2_MC14_100kEvts/merged_tmp_muons.root:variables


In [73]:
df_FSPs = pd.concat(dfs)

In [74]:
df_Y4S = fileY4S.arrays(library="pd")

In [75]:
print(df_FSPs.shape[0])
print(df_Y4S.shape[0])

5300642
116380


In [76]:
df_Y4Ssample = df_Y4S.sample(n=2000)
df_FSPssample = df_FSPs[df_FSPs['__event__'].isin(df_Y4Ssample["__event__"])]

In [77]:
df_Y4S=df_Y4Ssample
df_FSPs=df_FSPssample

In [78]:
print(df_FSPs.shape[0])
print(df_Y4S.shape[0])

48656
2000


## check if category combinations make sense

In [79]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
groupsAllFSPs

,basf2_used,basf2_Bsig,basf2_X,count
0,0.0,0.0,0.0,25981
1,0.0,1.0,0.0,274
2,1.0,0.0,0.0,4148
3,1.0,0.0,1.0,11052
4,1.0,1.0,0.0,7201


In [80]:
# function to create col with the particles mother B's uniqueParticleIdentifier
def B_ID(s):
    label = 0
    for i in range(10): 
        mcMotheri_uniqParID = "mcMother{}_uniqParID".format(i)
        if ((s[mcMotheri_uniqParID]) == 83886082.0):
            label = 83886082   
        elif ((s[mcMotheri_uniqParID]) == 83886081.0):
            label = 83886081   
    return label
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)


In [81]:
# if particle was used by basf2 but neither for B-sig or X it is from the Hc
def Hc(s):
    label = 0
    if ((s["basf2_used"] == 1.0) & (s["basf2_Bsig"] == 0.0) & (s["basf2_X"] == 0.0)):
            label = 1   
    
    return label
df_FSPs['Hc'] = df_FSPs.apply(Hc, axis=1)


In [82]:
# this shows that Hc is sometimes combined from both B's which is of course wrong
#groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","B_ID","Hc","basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(groupsAllFSPs.sort_values("Hc"))

## data preprocessing

### create df with per event info about which B is sig and which is tag

In [83]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby(["__event__","B_ID","Hc"]).size(),
                             'sum_p': df_FSPs.groupby(["__event__","B_ID","Hc"])["p"].sum()}).reset_index()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(groupsAllFSPs[(groupsAllFSPs["__event__"] == 3183239)].sort_values("__event__"))

Empty DataFrame
Columns: [__event__, B_ID, Hc, count, sum_p]
Index: []


In [85]:
print(groupsAllFSPs[groupsAllFSPs["Hc"] == 1].sort_values("__event__"))

      __event__      B_ID  Hc  count     sum_p
3         27852  83886082   1      3  2.200468
6         32984  83886081   1      3  2.178237
8         32984  83886082   1      1  0.188514
11        56383  83886081   1      2  0.723054
13        56383  83886082   1      2  0.466893
...         ...       ...  ..    ...       ...
6879   50644032  83886082   1      2  0.602594
6883   50673253  83886082   1      2  1.684088
6890   50780866  83886082   1      7  2.053427
6894   50885484  83886081   1      1  0.730875
6896   50885484  83886082   1      1  0.480291

[2150 rows x 5 columns]


In [86]:
events=[]
B_tag_IDs=[]
B_sig_IDs=[]

In [87]:
unclearHc = 0
for evt in pd.unique(groupsAllFSPs[groupsAllFSPs["Hc"] == 1]["__event__"]):
    # sort by sum_p to take "max_count_idx" from the H_c particles with more momentum if two categories have the same particle count
    singleEvt_Hcs = groupsAllFSPs[(groupsAllFSPs["Hc"] == 1) & (groupsAllFSPs["__event__"] == evt)].sort_values("sum_p",ascending=False)
    
    # B_ID=0 => background, so take the other one if available
    singleEvt_Hcs = singleEvt_Hcs[(singleEvt_Hcs["B_ID"] != 0)]
    
    if singleEvt_Hcs.empty:
        unclearHc += 1
        continue
    
    max_count_idx = singleEvt_Hcs["count"].idxmax()
    
    #print((max_count_idx))
    max_count_row = singleEvt_Hcs.loc[[max_count_idx]]
       
    
    #print(max_count_row,'\n\n\n\n')

        
    events.append(max_count_row.iloc[0]['__event__'])
    B_tag_IDs.append(max_count_row.iloc[0]['B_ID']) # this is Btag because it is Hc's mother B
    Bsig_tmp = 0
    if B_tag_IDs[-1] == 83886082:
        Bsig_tmp = 83886081.0
    elif B_tag_IDs[-1] == 83886081:
        Bsig_tmp = 83886082.0
    #else:
    #    unclearHc +=1
    #    print(singleEvt_Hcs)
    #    print(events[-1],B_tag_IDs[-1],max_count_idx)
    #    raise ValueError('Btag/Bsig assignment unclear')
    B_sig_IDs.append(Bsig_tmp)
    
print("in",unclearHc,"cases, Btag (B mother of Hc) was unclear")
print("equals to",round(unclearHc/(len(B_sig_IDs)+unclearHc) , 4)*100,"%")
    

in 4 cases, Btag (B mother of Hc) was unclear
equals to 0.27999999999999997 %


In [88]:
event_Bs = pd.DataFrame(
{"__event__" : events,
"B_tag_ID" : B_tag_IDs,
"B_sig_ID" : B_sig_IDs})

In [89]:
# throw away events with unclear Btag
print(df_FSPs.shape[0])
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(event_Bs["__event__"])]
print(df_FSPs.shape[0])

In [90]:
event_Bs[:10]

,__event__,B_tag_ID,B_sig_ID
0,27852.0,83886082.0,83886081.0
1,32984.0,83886081.0,83886082.0
2,56383.0,83886081.0,83886082.0
3,57846.0,83886082.0,83886081.0
4,127267.0,83886081.0,83886082.0
5,130678.0,83886081.0,83886082.0
6,134875.0,83886082.0,83886081.0
7,227910.0,83886082.0,83886081.0
8,419591.0,83886081.0,83886082.0
9,454733.0,83886081.0,83886082.0


In [91]:
# check that B-tag and B-sig are not equal for any event -> only 2 rows shall appear here
pd.DataFrame({'count' : event_Bs.groupby(["B_tag_ID","B_sig_ID"]).size()}).reset_index()

,B_tag_ID,B_sig_ID,count
0,83886081.0,83886082.0,719
1,83886082.0,83886081.0,704


### delete particles which occur more than ones based on uniqueParticleIdentifier

In [92]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(groupsFSPs_uniqParID.sort_values("count"))

In [93]:
print(df_FSPs.shape[0])
print(groupsFSPs_uniqParID.shape[0])
print(event_Bs.shape[0])

42251
34324
1423


In [94]:
# delete particles which occur more than ones (keep first) and if possible keep the one with basf2_used==1
df_FSPs_noDuplis = df_FSPs.sort_values("basf2_used",ascending=False).drop_duplicates(subset=("__event__","uniqueParticleIdentifier"), keep='first')

In [95]:
print(df_FSPs_noDuplis.shape[0])

34324


### save dataframes on NFS

In [96]:
df_FSPs_noDuplis.to_csv(root_path + "df_FSPs_preProcessed.csv")
event_Bs.to_csv(root_path + "event_Bs.csv")

### load dataframes

In [97]:
df_FSPs = pd.read_csv(root_path + "df_FSPs_preProcessed.csv")
event_Bs = pd.read_csv(root_path + "event_Bs.csv")

### delete Hc particles, after loading data, so saved df's cotain the Hc particles as well

In [98]:
print(df_FSPs.shape[0])
df_FSPs_final = df_FSPs[df_FSPs["Hc"] == 0]
print(df_FSPs_final.shape[0])

34324
30180


In [99]:
event_Bs.keys()

Index(['Unnamed: 0', '__event__', 'B_tag_ID', 'B_sig_ID'], dtype='object')

In [100]:
df_FSPs_final.keys()

Index(['Unnamed: 0', '__experiment__', '__run__', '__event__', '__candidate__',
       '__ncandidates__', '__weight__', 'basf2_X', 'basf2_used', 'basf2_Bsig',
       'isSignal', 'uniqueParticleIdentifier', 'mcErrors', 'mcPDG',
       'genMotherID', 'genMotherP', 'genMotherPDG', 'px', 'py', 'pz', 'pt',
       'p', 'E', 'kaonID', 'pionID', 'genMothPDG_0', 'genMothPDG_1',
       'genMothPDG_2', 'genMothPDG_3', 'genMothPDG_4', 'genMothPDG_5',
       'genMothPDG_6', 'genMothPDG_7', 'genMothPDG_8', 'genMothPDG_9',
       'mcMother0_uniqParID', 'mcMother1_uniqParID', 'mcMother2_uniqParID',
       'mcMother3_uniqParID', 'mcMother4_uniqParID', 'mcMother5_uniqParID',
       'mcMother6_uniqParID', 'mcMother7_uniqParID', 'mcMother8_uniqParID',
       'mcMother9_uniqParID', 'PDG', 'B_ID', 'Hc'],
      dtype='object')

# start of NN data creation

In [101]:
numFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["__event__"] ).size()}).reset_index()

minFSPs = numFSPs["count"].min()
maxFSPs = numFSPs["count"].max()
print("minFSPs:",minFSPs)
print("maxFSPs:",maxFSPs,'\n')

df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')

minFSPs: 4
maxFSPs: 49 



<ipython-input-101-887bf7860262>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')


In [105]:
data_dir = Path(nfs_path + "NNdata/" + data_subdir + root_subdir)    
data_dir.mkdir(parents=True, exist_ok=True)
print("Will save data to:", data_dir,'is', save_data ,'\n')

Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NNdata/Dstlnu_Hc_corr_BsigX_separation_dataRun1/axheim_data2_MC14_100kEvts is False 



In [102]:
for num_FSPs_toData in range(minFSPs, maxFSPs+1):
    df_num_subset = df_FSPs_final.copy()
    df_num_subset = df_num_subset[df_num_subset['numFSPs'] == num_FSPs_toData]
    
        
    numEvents = df_num_subset.__event__.nunique()
    print("numEvents:",numEvents)
    print("num_FSPs_toData:",num_FSPs_toData)  
    if numEvents == 0:
        print("skipped because empty \n")
        continue

    if numEvents < 10:
        print("skipped because <10 events \n")
        continue
    
    num_features = 4
    leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
    SA_target =  np.zeros((numEvents, num_FSPs_toData))
    global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)
    
    event_list = df_num_subset[df_num_subset["numFSPs"] == num_FSPs_toData]["__event__"].unique()
    #print("len(event_list):",len(event_list))
    for i in range(numEvents):

        event_iter = event_list[i]

        global_tag_masterInfo = "evt" + str(event_iter)
        global_tag[i,-1] = global_tag_masterInfo
        #print("global_tag[i,-1]:",global_tag[i,-1])
        #print("i:",i,"event_iter:",event_iter)

        event_df = df_num_subset[df_num_subset.__event__ == event_iter]

        for j in range(num_FSPs_toData):
            #print("numParticle:",j)
            particle = event_df.iloc[j]

            #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
            leaves[i,j,0] = particle["px"]
            leaves[i,j,1] = particle["py"]
            leaves[i,j,2] = particle["pz"]
            leaves[i,j,3] = particle["E"]
            
            basf2_usage = "basf2_NONE"
            if particle["basf2_Bsig"] == 1.0:
                basf2_usage = "basf2_Bsig"
            elif particle["basf2_X"] == 1.0:
                basf2_usage = "basf2_X"
            elif particle["basf2_used"] == 0:
                basf2_usage = "basf2_bg"

            global_tag_Info = str((particle["mcPDG"])) 
            global_tag_Info += "_" + basf2_usage
            global_tag[i,j] = global_tag_Info

            label = -10 # error code if assignment fails
            B_tag_uniqID = event_Bs[event_Bs.__event__ == event_iter].iloc[0]['B_tag_ID']
            B_sig_uniqID = event_Bs[event_Bs.__event__ == event_iter].iloc[0]['B_sig_ID']
            if particle["B_ID"] == B_tag_uniqID:
                label = 1 # particle belongs to X (MC truth)
            elif particle["B_ID"] == B_sig_uniqID:
                label = 2 # particle belongs to Bsig (MC truth)
            elif particle["B_ID"] == 0:
                label = 0 # background
            
            
            SA_target[i,j] = label
            
        del event_df
        
        
    # shuffle the data    
    for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
        perms = np.random.permutation(leaves.shape[1])

        leaves[idx,:] = leaves[idx,perms]
        SA_target[idx,:] = SA_target[idx,perms]
        global_tag[idx,0:-1] = global_tag[idx,perms]
        
        
         


    #print(global_tag)
    train_ratio = 0.75
    validation_ratio = 0.15
    test_ratio = 0.10

    print("leaves.shape:",leaves.shape)
    print("SA_target.shape:",SA_target.shape)
    print("global_tag.shape:",global_tag.shape)


    print("leaves[0]:",leaves[0])
    print("SA_target[0]:",SA_target[0])
    print("global_tag[0]:",global_tag[0])

    x=leaves
    y=SA_target
    z=global_tag

    x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
    x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 

    if save_data==True:
        np.save(data_dir / "leaves_train_{}_FSP{}.npy".format(mode,num_FSPs_toData), x_train)
        np.save(data_dir / "is_left_arr_train_{}_FSP{}.npy".format(mode,num_FSPs_toData), y_train)
        np.save(data_dir / "global_tag_train_{}_FSP{}.npy".format(mode,num_FSPs_toData), z_train)

        np.save(data_dir / "leaves_val_{}_FSP{}.npy".format(mode,num_FSPs_toData), x_val)
        np.save(data_dir / "is_left_arr_val_{}_FSP{}.npy".format(mode,num_FSPs_toData), y_val)
        np.save(data_dir / "global_tag_val_{}_FSP{}.npy".format(mode,num_FSPs_toData), z_val)

        np.save(data_dir / "leaves_test_{}_FSP{}.npy".format(mode,num_FSPs_toData), x_test)
        np.save(data_dir / "is_left_arr_test_{}_FSP{}.npy".format(mode,num_FSPs_toData), y_test)
        np.save(data_dir / "global_tag_test_{}_FSP{}.npy".format(mode,num_FSPs_toData), z_test)

    
    print("")
    #del df_num_subset


    del df_num_subset
                                          

numEvents: 2
num_FSPs_toData: 4
skipped because <10 events 

numEvents: 13
num_FSPs_toData: 5
leaves.shape: (13, 5, 4)
SA_target.shape: (13, 5)
global_tag.shape: (13, 6)
leaves[0]: [[ 1.31907356  0.95562345  0.71430713  1.84583934]
 [-0.52340865  0.06384798  0.5305444   0.89622961]
 [-1.0406692  -0.65080869 -0.01643937  1.23543292]
 [-0.06105278 -0.02850682  0.05907255  0.16586   ]
 [-0.14422755  1.67518437 -0.07337809  1.7538944 ]]
SA_target[0]: [2. 2. 2. 2. 1.]
global_tag[0]: [b'13.0_basf2_bg' b'-321.0_basf2_Bsig' b'211.0_basf2_Bsig'
 b'211.0_basf2_Bsig' b'-13.0_basf2_bg' b'evt4083146']
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NNdata/Dstlnu_Hc_corr_BsigX_separation/axheim_data2_MC14_100kEvts is False 


numEvents: 36
num_FSPs_toData: 6
leaves.shape: (36, 6, 4)
SA_target.shape: (36, 6)
global_tag.shape: (36, 7)
leaves[0]: [[-0.01811315 -0.05460083  0.07186858  0.50218662]
 [ 1.16328049 -1.14103985  0.8102603   1.88558536]
 [-1.0072813   1.14584816  0.

leaves.shape: (10, 17, 4)
SA_target.shape: (10, 17)
global_tag.shape: (10, 18)
leaves[0]: [[ 0.00882589 -0.07810738  0.09475593  0.12311517]
 [-0.04379231 -0.3893016   0.12385366  0.43392755]
 [ 0.07133107 -0.26045132 -0.13010611  0.29975091]
 [ 0.06132212 -0.12898847  0.02716882  0.14538423]
 [ 0.02872376  0.04855564 -0.00390706  0.05655059]
 [ 0.7155382   1.29805744  0.96194535  1.77250297]
 [-0.78095418  0.52676851  0.9567734   1.34991471]
 [-0.01006966  0.01170756  0.013218    0.02032684]
 [ 0.20043576 -0.24861896 -0.01891511  0.31991196]
 [-0.53669167 -0.37937599  0.62393022  0.91691488]
 [-0.89032865  0.21981518  0.41197103  1.01498957]
 [-0.01920847 -0.03866673  0.05213901  0.06769459]
 [-0.09180741 -0.03835562  0.06309391  0.18264854]
 [ 1.93600488  0.33516526  0.07708959  1.97126186]
 [-0.051345   -0.00902119  0.15210573  0.21291721]
 [-0.01246805  0.01305813 -0.01659379  0.02452185]
 [-0.10971558  0.0610147   0.07176651  0.14460544]]
SA_target[0]: [2. 1. 1. 1. 2. 1. 1. 0. 1. 

leaves.shape: (23, 22, 4)
SA_target.shape: (23, 22)
global_tag.shape: (23, 23)
leaves[0]: [[-1.57222245e-02  7.20205670e-03  2.53966823e-02  3.07253875e-02]
 [ 5.70950210e-01 -7.37419486e-01  9.40927386e-01  1.33213959e+00]
 [-9.70122889e-02  9.79806036e-02  1.82671919e-01  2.28868113e-01]
 [-1.63196027e-01  1.67178288e-02  1.78369939e-01  2.79657213e-01]
 [-1.21316276e-02 -1.49651961e-02 -2.04294045e-02  2.80801362e-02]
 [-4.12788123e-01  8.25173631e-02  2.77343124e-01  5.23070025e-01]
 [-4.09707986e-03  1.44639304e-02 -2.26767082e-02  2.72070660e-02]
 [-7.93932676e-02  4.82167229e-02  8.08436275e-02  1.86128256e-01]
 [ 4.62820470e-01 -8.73402283e-02  4.03879672e-01  6.35947942e-01]
 [-4.74212080e-01  4.16706532e-01  9.85877275e-01  1.27050872e+00]
 [ 3.34525496e-01 -2.65070945e-01 -8.16907361e-02  6.57693135e-01]
 [-3.75204831e-02  1.76470149e-02 -4.00667749e-02  5.76589130e-02]
 [-5.94656393e-02  9.69292130e-03  8.38325843e-02  1.03237673e-01]
 [-1.55027518e-02  1.89793129e-02 -6.08

leaves.shape: (44, 26, 4)
SA_target.shape: (44, 26)
global_tag.shape: (44, 27)
leaves[0]: [[ 3.53116319e-02 -2.71281004e-02  5.29153533e-02  6.91583675e-02]
 [ 3.81199792e-02  5.33321500e-02  1.23941191e-02  6.67163040e-02]
 [-7.18696341e-02 -8.96409452e-02  7.57488608e-02  1.96006443e-01]
 [-2.99317371e-02  4.07590158e-03 -4.47508767e-02  5.39922478e-02]
 [-1.07284121e-01 -4.93979782e-01  5.28082192e-01  7.44228864e-01]
 [-1.76966920e-01 -8.28228518e-02 -3.84658785e-03  1.95426999e-01]
 [ 6.93475688e-03 -2.36817859e-02  2.76450161e-02  3.70562377e-02]
 [-1.36648834e-01  3.93376127e-02 -2.64707476e-01  3.00483609e-01]
 [-1.43625541e-02 -2.21559152e-01  1.35447979e-01  2.60078634e-01]
 [-3.48779224e-02 -3.07773426e-02  2.39581373e-02  5.23230984e-02]
 [ 3.20164338e-02  2.09378108e-01  5.18369600e-02  2.58903857e-01]
 [-1.20138042e-02 -1.67267695e-02 -2.85958480e-02  3.52397337e-02]
 [-5.89205977e-03  3.65616102e-03  2.07988173e-02  2.19242943e-02]
 [-8.14416528e-01  6.74644768e-01  9.40

leaves.shape: (56, 30, 4)
SA_target.shape: (56, 30)
global_tag.shape: (56, 31)
leaves[0]: [[ 2.31666528e-02  2.09629778e-02  1.02901654e-02  3.28941901e-02]
 [ 1.52705923e-01 -1.89192593e-01 -5.83325140e-02  2.50031235e-01]
 [ 2.40280196e-01  4.16002721e-02  1.68747291e-01  2.96548147e-01]
 [-6.39157295e-02 -1.10196568e-01  5.44405915e-02  1.38536212e-01]
 [-1.35357920e-02 -7.28680706e-03 -1.74570605e-02  2.32607864e-02]
 [-9.23713520e-02  4.29207385e-02 -1.88673899e-01  2.14411978e-01]
 [ 3.06008523e-03  1.35514485e-02 -1.59003977e-02  2.11146519e-02]
 [-1.39185429e-01 -9.84629914e-02  3.14485610e-01  3.57727190e-01]
 [ 2.35443134e-02 -5.18639423e-02  1.32449064e-02  5.84776090e-02]
 [ 1.63342264e-02  1.28385350e-02  6.58843517e-02  6.90824343e-02]
 [ 1.14846036e-01  9.36759710e-02 -4.10181582e-02  1.53776750e-01]
 [-4.03503142e-02 -4.55572270e-03  1.65962148e-02  4.38672636e-02]
 [ 1.28043778e-02 -5.15716858e-02  5.99405207e-02  8.01027895e-02]
 [ 5.49174398e-02 -2.13828087e-02 -2.07

leaves.shape: (44, 33, 4)
SA_target.shape: (44, 33)
global_tag.shape: (44, 34)
leaves[0]: [[-2.28047725e-02  3.43933678e-03  8.51832796e-03  2.45855363e-02]
 [ 6.37107119e-02  4.38582897e-02  1.68799788e-01  1.85677066e-01]
 [ 3.23510244e-02 -7.00901821e-02  1.79806903e-01  1.95677655e-01]
 [ 3.45515199e-02 -3.32729965e-02  1.12652726e-01  1.85664830e-01]
 [-3.83136496e-02  5.47580481e-01  6.98761642e-02  5.70679323e-01]
 [ 7.50098348e-01 -4.14696075e-02  5.36033034e-01  9.22875221e-01]
 [-4.73508378e-03  1.88938193e-02 -1.79202016e-02  2.64675472e-02]
 [ 2.71018725e-02 -8.94608572e-02  3.28856893e-02  9.90920028e-02]
 [-1.42464377e-02 -3.67087483e-01  8.88517499e-01  1.08080364e+00]
 [-1.74001351e-01 -6.42339736e-02 -2.78648175e-02  2.33792245e-01]
 [-1.06373690e-01  2.48273332e-02  1.09491721e-01  1.54661551e-01]
 [-1.04324922e-01 -1.29602447e-01  1.44229382e-01  2.20187644e-01]
 [ 3.95920545e-01  2.40368396e-01 -2.52162844e-01  7.22380173e-01]
 [ 4.35983211e-01  1.86944788e-03  6.75

leaves.shape: (26, 36, 4)
SA_target.shape: (26, 36)
global_tag.shape: (26, 37)
leaves[0]: [[-1.11111557 -0.46356127  0.49095127  1.3001923 ]
 [ 0.00748283  0.41966912  0.1184758   0.65873489]
 [ 0.01304824  0.01300796 -0.01755518  0.02544893]
 [ 0.02775431  0.01210058 -0.00851519  0.0314521 ]
 [ 0.11075008  0.05673771  0.05315357  0.13531463]
 [ 0.19254382  0.16688351  0.00909551  0.25496266]
 [-0.168974    0.0828024  -0.07079104  0.20104681]
 [ 0.99477994 -0.79679412  0.2157418   1.30018938]
 [ 0.03299241  0.05163058 -0.03074613  0.06855319]
 [ 0.90413481  0.63810086  0.54078335  1.32694991]
 [-0.013876   -0.00792077 -0.02310598  0.02809214]
 [ 0.02394628  0.00691628  0.01694635  0.0301403 ]
 [ 0.18445873 -0.12837978 -0.01801806  0.22545741]
 [ 0.12111596  0.03879957  0.1113715   0.16905057]
 [ 0.11827747  0.0244561   0.05979792  0.19401893]
 [ 0.00189636 -0.01260962 -0.02110895  0.02466143]
 [-0.00376771  0.01817886 -0.01276578  0.02253068]
 [-0.07650327 -0.05048466  0.01724438  0.09

leaves.shape: (24, 39, 4)
SA_target.shape: (24, 39)
global_tag.shape: (24, 40)
leaves[0]: [[-6.46874076e-03  1.33258598e-02 -1.41686620e-02  2.04981494e-02]
 [ 1.82804550e-04 -1.91259012e-02  6.23187982e-03  2.01164072e-02]
 [ 1.73850730e-02  3.34696136e-02 -2.41773971e-03  3.77928732e-02]
 [ 1.57093499e-02  2.51054734e-01 -3.17205280e-01  4.04838799e-01]
 [-2.09042385e-01 -8.43092129e-02  1.83653995e-01  3.22514255e-01]
 [-6.55714348e-02 -1.57956496e-01  5.38606346e-02  1.79306541e-01]
 [ 5.95667720e-01  9.41337466e-01  3.33919615e-01  1.17128923e+00]
 [ 3.47698964e-02 -4.89808433e-03  2.39406750e-02  4.24981511e-02]
 [ 3.10842335e-01  1.60673246e-01  1.97772354e-01  4.25479313e-01]
 [-1.11356944e-01 -9.70367715e-02  4.60725203e-02  1.54722917e-01]
 [-1.24634840e-01  8.81057009e-02  1.00701414e-01  2.30037225e-01]
 [-7.18992576e-02 -5.30703962e-02 -6.08742842e-03  8.95713513e-02]
 [-1.07155807e-01  2.83939261e-02  6.27312064e-02  1.27372628e-01]
 [-1.61790494e-02 -8.67528468e-02  2.46